## Análise de Aprendizado Supervisionado dos Discursos do Senado

Este notebook estuda modelos de aprendizado supervisionado para classificar discursos de senadores por partido político.

## Inicializar e carregar dados

### Importar bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
from datasets import load_dataset_builder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

sns.set_theme(style='whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

### Carregar dados

In [ ]:
#DATA_PATH = Path('data/hf_discursos/data/full/discursos_2019-02-01_2023-01-31.parquet')
#df_raw = pd.read_parquet(DATA_PATH)

DATASET_HF_REPO = "fabriciosantana/discursos-senado-legislatura-56"
DATA_FILE_HF = {"train": "data/full/discursos_2019-02-01_2023-01-31.parquet"}

dataset = load_dataset(DATASET_HF_REPO, data_files=DATA_FILE_HF)
df_raw = dataset["train"].to_pandas()

df_raw.head()

## Preparar dados para análise

In [ ]:
colunas = ['Data', 'NomeAutor', 'Partido', 'UF', 'TextoDiscursoIntegral']
df = (

    df_raw[colunas]

    .rename(columns={

        'TextoDiscursoIntegral': 'texto',

        'NomeAutor': 'nome_autor'

    })

    .dropna(subset=['Partido', 'texto'])

)

df['texto'] = df['texto'].str.strip()
df = df[df['texto'].str.len() > 0]
df['n_palavras'] = df['texto'].str.split().str.len()
df = df[df['n_palavras'] >= 20]
df

## Discursos por partido

In [ ]:
top_partidos = df['Partido'].value_counts().head(8)
top_partidos

## Selecionar partidos

In [ ]:
partidos_selecionados = top_partidos.index.tolist()
df_modelo = df[df['Partido'].isin(partidos_selecionados)].copy()
max_por_partido = 800
df_balanceado = (
    df_modelo
        .groupby('Partido', group_keys=False)
        .apply(lambda grupo: grupo.sample(min(len(grupo), max_por_partido), random_state=42))
        .reset_index(drop=True)    
    )

df_balanceado['Partido'].value_counts()

## Separar conjunto de treino e teste

In [ ]:
X = df_balanceado['texto']
y = df_balanceado['Partido']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
len(X_train), len(X_test)

## Aplicar Regressão Logística com TF-IDF

In [ ]:
pipeline_lr = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=20000,
        ngram_range=(1, 2),
        min_df=5,
        strip_accents='unicode'
    )),
    ('clf', LogisticRegression(
        max_iter=1000,
        #multi_class='multinomial',
        solver='lbfgs',
        random_state=42
    ))
])
pipeline_lr.fit(X_train, y_train)
y_pred_lr = pipeline_lr.predict(X_test)
acc_lr = accuracy_score(y_test, y_pred_lr)
print(f'Acurácia (logística): {acc_lr:.3f}')
print(classification_report(y_test, y_pred_lr)) 

In [ ]:
cm_lr = confusion_matrix(y_test, y_pred_lr, labels=partidos_selecionados)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_lr, display_labels=partidos_selecionados)
disp.plot(cmap='Blues', xticks_rotation=45)
plt.title('Matriz de confusão — Regressão Logística')
plt.tight_layout()

## Aplicar SVM Linear

In [ ]:
pipeline_svm = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=20000,
        ngram_range=(1, 2),
        min_df=5,
        strip_accents='unicode'
    )),
    ('clf', LinearSVC(random_state=42))
])
pipeline_svm.fit(X_train, y_train)
y_pred_svm = pipeline_svm.predict(X_test)
acc_svm = accuracy_score(y_test, y_pred_svm)
print(f'Acurácia (SVM linear): {acc_svm:.3f}')
print(classification_report(y_test, y_pred_svm))

## Comparar desempenho

In [ ]:
pd.DataFrame({
    'modelo': ['Regressão Logística', 'SVM Linear'],
    'acuracia': [acc_lr, acc_svm]
})

Os resultados indicam que modelos lineares com TF-IDF capturam bem o padrão de vocabulário associado a cada partido, tornando possível distinguir discursos com boa acurácia.